In [51]:
!pip install sentence-transformers llama-index llama-index-embeddings-huggingface datasets llama-index-llms-huggingface

  Obtaining dependency information for llama-index-llms-huggingface from https://files.pythonhosted.org/packages/b3/0e/ec2114b8e0a26bdfec30432a95f49a038cf07a237737e4724e3430562564/llama_index_llms_huggingface-0.3.5-py3-none-any.whl.metadata
  Obtaining dependency information for text-generation<0.8.0,>=0.7.0 from https://files.pythonhosted.org/packages/7b/79/8fc351fd919a41287243c998a47692c7eb0fa5acded13db0080f2c6f1852/text_generation-0.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for accelerate>=0.26.0 from https://files.pythonhosted.org/packages/d1/d5/0050b2820a1e709ffa623f9a9e8ae42d0903535f2150613cbfeb7f16932a/accelerate-1.1.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 5.2 MB/s eta 0:00:00a 0:00:01


In [ ]:


from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

from transformers import AutoTokenizer
import torch
from llama_index.llms.huggingface import HuggingFaceLLM

nr_llama3 = "NousResearch/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(
    nr_llama3
)

stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

llm = HuggingFaceLLM(
    model_name=nr_llama3,
    model_kwargs={
        "torch_dtype": torch.bfloat16,  # comment this line and uncomment below to use 4bit
        # "quantization_config": quantization_config
    },
    generate_kwargs={
        "do_sample": True,
        "temperature": 0.6,
        "top_p": 0.9,
    },
    tokenizer_name=nr_llama3,
    stopping_ids=stopping_ids,
)

from llama_index.core import Settings

# bge embedding model
Settings.embed_model = embed_model

# Llama-3-8B-Instruct model
Settings.llm = llm


from datasets import load_dataset
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

from llama_index.core import VectorStoreIndex, Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core import Settings
from llama_index.llms.huggingface import HuggingFaceLLM
import tiktoken

# wkds = load_dataset("google/wiki40b", "en", split="train", streaming=True)
wkds = load_dataset("microsoft/wiki_qa", split="train", streaming=True)
wkitr = iter(wkds)
recs = []

for i in range(10):
  recs.append(next(wkitr))

processed_data_list = [rec['answer'] for rec in recs]


from llama_index.core.node_parser import SentenceSplitter

Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained("HuggingFaceH4/zephyr-7b-alpha")

splitter = SentenceSplitter(
    chunk_size=50,
    chunk_overlap=5,
    tokenizer=tokenizer
)

# paragraph_separator="_START_PARAGRAPH_", 
    

lngdocs = [Document(text=wktxt) for wktxt in processed_data_list]
nodes = splitter.get_nodes_from_documents(lngdocs)

index = VectorStoreIndex(nodes)

query_str = "give me an example of a circular motion?"

query_engine = index.as_query_engine(similarity_top_k=2, llm=llm)# use this for testingvector_retriever = index.as_retriever(similarity_top_k=2)

response = query_engine.query(query_str)
print(response)
